# Otimização com conhecimento prévio dos valores de ganhos PD
Otimização em lote do kuka utilizando conhecimento prévio dos ganhos de um controlador PD.

In [1]:
include("../comum.jl")
include("../modelos.jl")
using Evolutionary, Plots;
pyplot();

In [2]:
Ts = 0.08
tend = 2.0
t0 = 0.0
popul  = 10   # população
iterac = 3 #iterações
xr = SVector{7}([1.,1.,0.5,0.6,0.5,1.2,0.8]);

In [3]:
function gerador(n)
    rand(n).*[10000.,10000.,10000.,10000.,1000.,1000.,100.,1000.,10000.,1000.,1000.,1000.,100.,10.]
end;

In [4]:
function custo(gain::Vector{Float64})
    kp = SMatrix{7,7}(diagm(gain[1:7]))
    kv = SMatrix{7,7}(diagm(gain[8:end]))
    x, v, t, a, ta, j, tj = kukaRobot(kp, kv, Ts, t0, tend, xr)
    sizeVector = length(x[1])

    erro_sum = 0.
    for i=1:7
        erro = -(x[i]-xr[i])
        erro_sum = erro_sum + sum(abs.(erro[floor(Integer,sizeVector/3):end])) 
    end

    jerk_sum = 0.
    for i=1:7
        jerk_sum = jerk_sum + sum(abs.(j[i]))
    end

    
    erro_sum = erro_sum * 10.
    jerk_sum = jerk_sum * 0.01

    out =  erro_sum + jerk_sum
    println("erro: $(erro_sum) jerk: $(jerk_sum)")
    out
end;

In [ ]:
N = 14
result, fitness, cnt = ga(custo, N; initPopulation = gerador, populationSize = popul, ɛ = 0.25, selection = roulette, crossover = intermediate(0.25), mutation = domainrange(fill(0.5,N)), iterations = iterac, showBar = true);

In [ ]:
t_end_new = tend
kp = SMatrix{7,7}(diagm(result[1:7]))
kv = SMatrix{7,7}(diagm(result[8:end]))

x, v, t, a, ta, j, tj = kukaRobot(kp, kv, Ts, t0, t_end_new, xr);

In [ ]:
for i=1:7
    erro = xr[i] - x[i][end]
    println("erro final $(i) = $(erro)")
end   

In [ ]:
function plotx(i)
    p1 = plot(t,x[i], label = "desejado")
    p1= plot!([xr[i]],seriestype= :hline, label = "referência");
end    
function plotx()
    p1 = plot(t,x[1], label = "simulação 1")
    p1= plot!([xr[1]],seriestype= :hline, label = "referência 1")
    p2 = plot(t,x[2], label = "simulação 2")
    p2= plot!([xr[2]],seriestype= :hline, label = "referência 2")
    p3 = plot(t,x[3], label = "simulação 3")
    p3= plot!([xr[3]],seriestype= :hline, label = "referência 3")
    p4 = plot(t,x[4], label = "simulação 4")
    p4= plot!([xr[4]],seriestype= :hline, label = "referência 4")
    p5 = plot(t,x[5], label = "simulação 5")
    p5= plot!([xr[5]],seriestype= :hline, label = "referência 5")
    p6 = plot(t,x[6], label = "simulação 6")
    p6= plot!([xr[6]],seriestype= :hline, label = "referência 6")
    p7 = plot(t,x[7], label = "simulação 7")
    p7= plot!([xr[7]],seriestype= :hline, label = "referência 7")
    plot(p1,p2,p3,p4,p5,p6,p7, layout =(7,1))
end;

In [ ]:
plotx()

In [ ]:
println("Somatório do módulo do jerk")
for i = 1:7
    println("junta $(i) -> $(sum(abs.(j[i])))")
end
soma_jerk= 0.
for i = 1:7
    soma_jerk += sum(abs.(j[i]))
end
println("jerk total: $(soma_jerk)")

In [ ]:
println("Máximo módulo do jerk")
for i = 1:7
    println("junta $(i) -> $(maximum(abs.(j[i])))")
end 
soma_jerk_max = 0.
for i = 1:7
    soma_jerk_max += maximum(abs.(j[i]))
end
println("jerk máximo total: $(soma_jerk_max)")